In [1]:
import math, os, json, sys, re
from glob import glob
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd

import PIL
from PIL import Image
from numpy.random import random, permutation, randn, normal, uniform, choice
from numpy import newaxis
import scipy
from scipy import misc, ndimage
from scipy.ndimage.interpolation import zoom
from scipy.ndimage import imread
from sklearn.metrics import confusion_matrix

import keras
from keras import backend as K
from keras.utils.data_utils import get_file
from keras.utils import np_utils
from keras.utils.np_utils import to_categorical
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Reshape, merge, LSTM, Bidirectional
from keras.layers import TimeDistributed, Activation, SimpleRNN, GRU
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.layers.normalization import BatchNormalization
from keras.optimizers import SGD, RMSprop, Adam
from keras.metrics import categorical_crossentropy, categorical_accuracy
from keras.layers.convolutional import *
from keras.preprocessing import image, sequence
from keras.preprocessing.text import Tokenizer
from keras.callbacks import TensorBoard, EarlyStopping, ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint

import matplotlib.pyplot as plt
import matplotlib.patheffects as PathEffects
import matplotlib

from alexnet import AlexNet_base
import theano

keras.backend.set_image_dim_ordering('th')

/home/okon/anaconda2/envs/newtf/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


In [ ]:
#keras.backend.set_image_dim_ordering('th')

In [2]:
train_dir = '/triplet-loss-with-svc/alexnet/train/'
test_dir = '/triplet-loss-with-svc//alexnet/test1/'

In [3]:
# Function for generating batches
def get_batches(dirname, gen=image.ImageDataGenerator(data_format='channels_first'), shuffle=False, batch_size=32, class_mode='categorical',
                target_size=(227,227)):
    return gen.flow_from_directory(dirname, target_size=target_size,
            class_mode=class_mode, shuffle=shuffle, batch_size=batch_size)

# Function for getting classes
def get_classes():
    batches = get_batches(train_dir, shuffle=False, batch_size=1)
    test_batches = get_batches(test_dir, shuffle=False, batch_size=1)
    return (batches.classes,onehot(batches.classes),batches.filenames, test_batches.filenames)

# Load all data
def get_data(path, target_size=(227,227)):
    batches = get_batches(path, shuffle=False, batch_size=1, class_mode=None, target_size=target_size)
    return np.concatenate([batches.next() for i in range(batches.samples)])

def onehot(x):
    return to_categorical(x)

In [4]:
# Creating a data generator for training as it is more economical a=in terms of space utility
train_gen = get_batches(train_dir, batch_size=32)
# Loading the data for using whole data set to perform validation as well
train_img = get_data(train_dir) 

Found 22424 images belonging to 10 classes.
Found 22424 images belonging to 10 classes.


In [5]:
train_img.shape

(22424, 3, 227, 227)

In [6]:
(trn_classes,trn_labels, filenames, test_filenames) = get_classes()

Found 22424 images belonging to 10 classes.
Found 79726 images belonging to 1 classes.


In [7]:
# Initializing Alexnet model
# import gc
# gc.collect()
inp = Input(shape=(3, 227, 227))
weights_path = './weights/alexnet_weights.h5'
alexnet = AlexNet_base(input_tensor=inp, weights_path=weights_path)

x = Dropout(0.4)(alexnet.get_output_at(-1))
x = Dense(10, activation='softmax', name='output')(x)
sm_model = Model(inp, x)

/home/okon/anaconda2/envs/newtf/lib/python2.7/site-packages/keras/layers/core.py:630: UserWarning: `output_shape` argument not specified for layer mean_subtraction and cannot be automatically inferred with the Theano backend. Defaulting to output shape `(None, 3, 227, 227)` (same as input shape). If the expected output shape is different, specify it via the `output_shape` argument.
  .format(self.name, input_shape))
alexnet.py:30: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(96, (11, 11), strides=(4, 4), activation="relu", name="conv_1")`
  name='conv_1')(mean_subtraction)
alexnet.py:38: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), activation="relu", name="conv_2_1")`
  ) for i in range(2)], mode='concat',concat_axis=1,name="conv_2")
alexnet.py:38: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (5, 5), activation="relu", name="conv_2_2")`
  ) for i in range(2)], mode='concat',concat_axis=1,name="conv_2")


In [8]:
sm_model.compile(optimizer=keras.optimizers.Adam(lr=0.0001, decay=0.005), loss='categorical_crossentropy', 
                 metrics=['accuracy'])

In [9]:
sm_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 3, 227, 227)  0                                            
__________________________________________________________________________________________________
mean_subtraction (Lambda)       (None, 3, 227, 227)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv_1 (Conv2D)                 (None, 96, 55, 55)   34944       mean_subtraction[0][0]           
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 96, 27, 27)   0           conv_1[0][0]                     
__________________________________________________________________________________________________
convpool_1

In [10]:
def get_callbacks(res_path):
    es = EarlyStopping(patience=4, verbose=1, monitor='val_loss')
    rlrp = ReduceLROnPlateau(factor=0.5, patience=2, verbose=1, monitor='val_loss')
    ckp = ModelCheckpoint(model_path+'alexnet_trained_model.hdf5', monitor='val_loss', mode = 'min', save_best_only=True, verbose=1)
    return es, rlrp, ckp

In [11]:
ver = 'benchmark_V6'
model_path = 'data/models/'+ver+'/'
if not os.path.exists(model_path):
    os.makedirs(model_path)

In [12]:
# For trainig look at the accuracy and run for 3-3 epochs for better convergence
es, rlrp, ckp = get_callbacks(model_path)
sm_model.fit(train_img, trn_labels ,batch_size=64, epochs=15, verbose=1)
             #callbacks=[es, rlrp, ckp])

Epoch 1/15
22424/22424 [==============================] - 1281s 57ms/step - loss: 0.2840 - acc: 0.9130
Epoch 2/15
22424/22424 [==============================] - 1316s 59ms/step - loss: 0.0216 - acc: 0.9947
Epoch 3/15
22424/22424 [==============================] - 1533s 68ms/step - loss: 0.0081 - acc: 0.9982
Epoch 4/15
22424/22424 [==============================] - 1527s 68ms/step - loss: 0.0042 - acc: 0.9992
Epoch 5/15
22424/22424 [==============================] - 1532s 68ms/step - loss: 0.0021 - acc: 0.9999
Epoch 6/15
22424/22424 [==============================] - 1526s 68ms/step - loss: 0.0016 - acc: 0.9999
Epoch 7/15
22424/22424 [==============================] - 1524s 68ms/step - loss: 0.0014 - acc: 0.9998
Epoch 8/15
22424/22424 [==============================] - 1521s 68ms/step - loss: 0.0012 - acc: 0.9999
Epoch 9/15
22424/22424 [==============================] - 1521s 68ms/step - loss: 7.4418e-04 - acc: 1.0000
Epoch 10/15
22424/22424 [==============================] - 1519s 68ms

In [13]:
sm_model.save_weights(model_path+'alexnet_trained_model.hdf5')

In [14]:
sm_model.load_weights(model_path+'alexnet_trained_model.hdf5')
sm_model.evaluate(train_img, trn_labels, batch_size=64)

22424/22424 [==============================] - 477s 21ms/step


[0.00011943863339129874, 1.0]

In [15]:
# Test generator(put the test folder in test_all folder)
test_imgs = get_batches(test_dir, batch_size=64)
pred_res = sm_model.predict_generator(test_imgs, verbose=1)

Found 79726 images belonging to 1 classes.
1246/1246 [==============================] - 3888s 3s/step


In [16]:
classes = sorted(train_gen.class_indices, key=train_gen.class_indices.get)

In [17]:
#def do_clip(arr, mx): return np.clip(arr, (1-mx)/9, mx)
def do_clip(arr, mx): return np.clip(arr, 0, 1)

#sub = do_clip(pred_res,0.93)
sub = do_clip(pred_res,1)

In [18]:
classes = sorted(train_gen.class_indices, key=train_gen.class_indices.get)
submission = pd.DataFrame(sub, columns=classes)
submission.insert(0, 'img', [a[5:] for a in test_filenames])
submission.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_1.jpg,0.007778,0.007778,0.007778,0.007778,0.007778,0.930000,0.007778,0.007778,0.007778,0.007778
1,img_10.jpg,0.007778,0.007778,0.007778,0.007778,0.007778,0.930000,0.007778,0.007778,0.007778,0.007778
2,img_100.jpg,0.072103,0.007778,0.007778,0.042754,0.007778,0.007778,0.007778,0.007778,0.007778,0.878685
3,img_1000.jpg,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.930000,0.007778
4,img_100000.jpg,0.007778,0.007778,0.007778,0.011136,0.007778,0.007778,0.007778,0.007778,0.019168,0.930000


In [19]:
submission.to_csv(model_path+'softmax_sub'+ver+'.csv', index=False)

In [20]:
model_path+'sub'+ver+'.csv'

'data/models/benchmark_V6/subbenchmark_V6.csv'

In [21]:
from sklearn.svm import SVC

In [22]:
# Input features to resnet
alexnet_feats = alexnet.predict(train_img, batch_size=64, verbose=1)

22424/22424 [==============================] - 480s 21ms/step


In [23]:
clf=SVC(probability=True, verbose=True, kernel='rbf')
clf.fit(alexnet_feats, trn_classes)

[LibSVM]

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=True, random_state=None, shrinking=True,
  tol=0.001, verbose=True)

In [24]:
# Test Input features to Resnet
pred_test_res = alexnet.predict_generator(test_imgs, verbose=1)
pred = clf.predict_proba(pred_test_res)
pred.shape

1246/1246 [==============================] - 3354s 3s/step


(79726, 10)

In [25]:
sub = do_clip(pred,0.93)
classes = sorted(train_gen.class_indices, key=train_gen.class_indices.get)
submission = pd.DataFrame(sub, columns=classes)
submission.insert(0, 'img', [a[5:] for a in test_filenames])
submission.head()

,img,c0,c1,c2,c3,c4,c5,c6,c7,c8,c9
0,img_1.jpg,0.007778,0.007778,0.007778,0.007778,0.007778,0.930000,0.007778,0.007778,0.007778,0.007778
1,img_10.jpg,0.007778,0.007778,0.007778,0.007778,0.007778,0.930000,0.007778,0.007778,0.030901,0.011270
2,img_100.jpg,0.134142,0.021183,0.007778,0.096222,0.007822,0.007778,0.007778,0.007778,0.010538,0.724275
3,img_1000.jpg,0.007778,0.010063,0.007778,0.007778,0.007778,0.007778,0.007778,0.007778,0.930000,0.007778
4,img_100000.jpg,0.009578,0.007778,0.007778,0.094278,0.023633,0.020791,0.007778,0.007778,0.054774,0.795840


In [26]:
submission.to_csv(model_path+'svm_sub'+ver+'svm.csv', index=False)